## Google BigData Showcase
BigQuery ML Tutorial 02  
Edited by 김하제

In [2]:
from google.cloud import bigquery
client = bigquery.Client()

In [4]:
dataset = bigquery.Dataset(client.dataset('bqml_tutorial'))
#dataset.location = 'US'
#client.create_dataset(dataset)

In [5]:
%load_ext google.cloud.bigquery

In [7]:
%%bigquery
CREATE OR REPLACE MODEL bqml_tutorial.ncaa_model OPTIONS(
  model_type='linear_reg', max_iteration=50) AS
SELECT
  * EXCEPT(
    game_id, season, scheduled_date,
    total_three_points_made,
    total_three_points_att),
  total_three_points_att as label
FROM
  bqml_tutorial.wide_games
WHERE
  # remove the game to predict
  game_id != 'f1063e80-23c7-486b-9a5e-faa52beb2d83'

""


In [8]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_tutorial.ncaa_model`)

,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,9,46.676759,97.574771,9169,0.10
1,0,8,54.230652,117.709800,11028,0.05
2,0,7,64.009904,119.191259,9454,0.05
3,0,6,77.645875,143.565330,7795,0.05
4,0,5,96.821618,148.159515,9193,0.05
5,0,4,124.060249,193.830190,9208,0.05
6,0,3,163.280971,207.578136,14012,0.05
7,0,2,221.010915,296.070452,7762,0.05
8,0,1,310.393369,339.965668,7333,0.05
9,0,0,524.281301,587.488959,6357,0.20


In [9]:
%%bigquery
WITH eval_table AS (
SELECT
  *,
  total_three_points_att AS label
FROM
  `bqml_tutorial.wide_games` )
SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_tutorial.ncaa_model`,
    TABLE eval_table)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,5.713375,57.099564,0.035362,4.512607,0.245905,0.414163


In [10]:
%%bigquery
  WITH game_to_predict AS (
  SELECT
    *
  FROM
    `bqml_tutorial.wide_games`
  WHERE
    game_id='f1063e80-23c7-486b-9a5e-faa52beb2d83' )
SELECT
  truth.game_id AS game_id,
  total_three_points_att,
  predicted_total_three_points_att
FROM (
  SELECT
    game_id,
    predicted_label AS predicted_total_three_points_att
  FROM
    ML.PREDICT(MODEL `bqml_tutorial.ncaa_model`,
      table game_to_predict) ) AS predict
JOIN (
  SELECT
    game_id,
    total_three_points_att AS total_three_points_att
  FROM
    game_to_predict) AS truth
ON
  predict.game_id = truth.game_id

,game_id,total_three_points_att,predicted_total_three_points_att
0,f1063e80-23c7-486b-9a5e-faa52beb2d83,50,48.631394
